# preamble

In [7]:
########## PREAMBLE ###########################################################

# general imports
import openai
import os
import requests
import json

# import scripts
from scripts import transcribe, process_transcript, check_response, prepare_yield_NOTION
from prompts import prompts

# declaring secrets
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
NOTION_API_KEY = os.environ["NOTION_API_KEY"]
NOTION_DATABASE_ID = os.environ["NOTION_DATABASE_ID"]

# declaring static variables
MAX_WHISPER_AUDIO_SIZE = 26262828

MAX_CONTEXT_4K = 4097
MAX_CONTEXT_16K = 4097 * 4

MAX_BLOCK_SIZE = 2000

# declaring dynamic variables
AUDIO_FILE_PATH = "./to_process/230922-AI_interview-Don.mp3"
THOUGHT = "interview_english"

In [8]:
# print all Thought options
for thought in prompts:
    print(thought)

daily_reflection
shower
blog
interview_nederlands
interview_english


# thought processing

### transcription

In [18]:
print("Transcription started.")
transcript = transcribe(THOUGHT, AUDIO_FILE_PATH) # also deals with long audio files
print("Transcription finished!")

Transcription started.
Transcription finished!


In [19]:
transcript

"It still works well with cars and wind and such. But okay, so I'm working on a project regarding AI. I can tell you more about it after the conversation. But I would like to know more about how people feel about AI and what it means to them. So I'm not looking for any specific answers. So just feel free to talk and say whatever comes to your mind. But just as a leading question, do you have a definition of AI for yourself? And how do you feel about it? Good question. I view AI really from the machine learning perspective. So to me it's basically, as also with the physicist background, it's like a fancy multi-parameter fitting. So basically you give it a lot of options, you train it, and then it will find a good solution. But it also seems to me really like a black box. I think that also holds true for most people, that they most of the times don't really know what it's doing. And it works, so that's nice. You can prove that it works, so it's not really a problem. But what really happe

### transcript processing

In [20]:
print("Text processing started.")
text_response = process_transcript(THOUGHT, transcript)
print("Text processing succes!")

Text processing started.
Text processing succes!


In [21]:
print(type(text_response["choices"][0]["message"]["content"]))
text_response["choices"][0]["message"]["content"]

<class 'str'>


'{"title": "Opinions on AI and Its Impact", "summary": "The interviewee views AI as a black box and thinks it is overused in many cases. They appreciate its capabilities and are amazed by its applications in chat GPT, art generation, and music generation. The interviewee uses chat GPT for tasks like writing abstracts. They are not super afraid of AI and believe that computers are limited by boundaries. The need for special regulations depends on the application. They mention copyright issues in art and music and discuss the challenge of generative models creating biased content. The interviewee mentions the potential bias in profiling people with AI. They believe that citizens can influence the future of AI through regular politics and by their actions.", "opinion on AI": "The interviewee views AI as a black box and thinks it is overused in many cases.", "ideas": ["AI as a fancy multi-parameter fitting", "AI as a black box", "Overuse of AI", "Appreciation for AI\'s capabilities", "Appl

### check data Notion

In [33]:
content_notion = check_response(text_response)
content_notion

{'title': 'Opinions on AI and Its Impact',
 'summary': 'The interviewee views AI as a black box and thinks it is overused in many cases. They appreciate its capabilities and are amazed by its applications in chat GPT, art generation, and music generation. The interviewee uses chat GPT for tasks like writing abstracts. They are not super afraid of AI and believe that computers are limited by boundaries. The need for special regulations depends on the application. They mention copyright issues in art and music and discuss the challenge of generative models creating biased content. The interviewee mentions the potential bias in profiling people with AI. They believe that citizens can influence the future of AI through regular politics and by their actions.',
 'opinion on AI': 'The interviewee views AI as a black box and thinks it is overused in many cases.',
 'ideas': "- AI as a fancy multi-parameter fitting\n- AI as a black box\n- Overuse of AI\n- Appreciation for AI's capabilities\n- Ap

In [32]:
keys = [key for key, content in content_notion.items()]
keys

['title', 'summary', 'opinion on AI', 'ideas', 'topics']

### upload to Notion

In [34]:
url = "https://api.notion.com/v1/pages/"

headers =   {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json"
            }

data_database = prepare_yield_NOTION(THOUGHT, NOTION_DATABASE_ID, MAX_BLOCK_SIZE,
                                     content_notion, transcript)

response_get = requests.post(url, headers=headers, json=data_database)
print("Processed text uploading succesful!")

Processed text uploading succesful!


In [35]:
response_get.json()

{'object': 'page',
 'id': '0cc1023f-6acf-45d3-ae75-2a7f36674861',
 'created_time': '2023-09-23T21:58:00.000Z',
 'last_edited_time': '2023-09-23T21:58:00.000Z',
 'created_by': {'object': 'user',
  'id': 'dc664599-409b-4fbc-b638-fa34c238bb19'},
 'last_edited_by': {'object': 'user',
  'id': 'dc664599-409b-4fbc-b638-fa34c238bb19'},
 'cover': None,
 'icon': {'type': 'emoji', 'emoji': '🎙️'},
 'parent': {'type': 'database_id',
  'database_id': 'e5568bc0-5015-4ec7-bd8c-ba5050353de6'},
 'archived': False,
 'properties': {'Archive': {'id': '%3Fohe',
   'type': 'checkbox',
   'checkbox': False},
  'Origin': {'id': 'BAiK',
   'type': 'created_time',
   'created_time': '2023-09-23T21:58:00.000Z'},
  'Thought': {'id': 'FIsG',
   'type': 'select',
   'select': {'id': '43331476-f532-431a-af65-54d7c8f768d1',
    'name': 'interview_english',
    'color': 'pink'}},
  'Public': {'id': 'a~gd', 'type': 'checkbox', 'checkbox': False},
  'Name': {'id': 'title',
   'type': 'title',
   'title': [{'type': 'text'

---

# Testing

In [13]:
def is_jsonable(x):
    try:
        json.dumps(x)
        return True
    except (TypeError, OverflowError):
        return False

In [40]:
for key in data_database:
    print(is_jsonable(data_database[key]))

True
True
True
True
